# <center> Apprentissage statistique <center>
## <center> Projet <center>
### <center> Bounds on the prediction error of penalized least squares estimators with convex penalty <center>

In [ ]:
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
rng = default_rng(0)


n_samples = 1000
n_features = 100
n_informative = 10
bias = 1
noise = 2

dataset = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative, bias=bias, noise=noise, 
                          coef=True, random_state=0)

dataset, coef = dataset[:2], dataset[2]

# set the parameters of the linear model with the values of those used by sklearn to create the dataset

lr = LinearRegression()
lr.coef_ = coef
lr.intercept_ = bias

f = lr.predict(dataset[0])  # the mean vector, that we can compute thanks to the coef given by make_regression

added_noise = dataset[1] - f  # noise added by sklearn function make_regression, i.e y - f with the paper notations

X_train, X_test, y_train, y_test = train_test_split(*dataset, test_size=0.2, train_size=0.8, random_state=0, shuffle=True, stratify=None)

## Ordinary least square estimator

In [ ]:
# compute the restriction of f on the test set (needed because it has been shuffled by train_test_split)

lr = LinearRegression()

lr.coef_ = coef
lr.intercept_ = bias

f_test = lr.predict(X_test)

# fit the model and make predictions

lr.fit(X_train, y_train) # Note that fitting again the model erase the previous weights
predictions = lr.predict(X_test)

# compute the error (MSE) between our predictor and the vector f

error = MSE(f_test, predictions) 

print(f"MSE for OLS regression {error}\n")
print(f"l2-norm of the weights for OLS model : {np.linalg.norm(lr.coef_)}")
print(f"l1-norm of the weights for OLS model : {np.linalg.norm(lr.coef_, ord=1)}")

print(f"\n\tMeasure the spreadness inside the OLS vector of weights")

coeff_OLS = pd.Series(lr.coef_)
coeff_OLS.describe()

## Lasso estimator

In [ ]:
# now we fit the lasso estimator with different alpha parameter (plays the role of lambda here) and see if it performs better

all_alphas = np.arange(10) / 100

for alpha in all_alphas:

    lasso = Lasso(alpha=alpha)

    lasso.fit(X_train, y_train)

    predictions = lasso.predict(X_test)

    error = MSE(f_test, predictions) 
    
    if alpha == 0.1:
        
        print("\t\t Measure the spreadness inside the Lasso vector of weights\n")
        coeff_lasso = pd.Series(lasso.coef_)
        print(np.where(lasso.coef_ == np.max(lasso.coef_)))
        print(coeff_lasso.describe())
        print()
    
    print(f"MSE of Lasso : {error} with alpha = {alpha}")
    print(f"l2-norm of the weights for Lasso model : {np.linalg.norm(lasso.coef_)}")
    print(f"l1-norm of the weights for OLS model : {np.linalg.norm(lasso.coef_, ord=1)}\n")

With a sample of n = 10000

With 10000 features :

With only 10 informative features :

This is a classical case were simple linear regression will perform bad. There are so few informative features, OLS will catch a lot of noise and is prompt to overfitting. It has a MSE of 6205.81 and the l2-norm of the weights is 160.32.
With $\alpha = 0.1$, lasso is so much better with a loss of only 0.08. It can be 1 or 2 order of magnitude bigger when $\alpha$ is greater, but it is still far below the OLS MSE. 
Curiously, the weights norm is bigger (6046.88) for the lasso than for the OLS. Probabily the Lasso was able to select the usefull variables and putting big weights on them was still achieving better results and a better optimization of the regularized cost function than what the OLS was able to achieve on the original cost function. 
This last intuition is in fact confirmed when one prints the l1-norm, and it totally makes sense. It is huge for OLS (6046.88), but for the lasso with $\alpha = 0.1$, it is much lower (478.13), even if the l2-norm of the weights is really close to the l2-norm of OLS weights. It can only be the case if many coefficients of the lasso are shrinked towards zero, such that taking the square makes them even smaller. So even if some weights of the lasso are still big, the others are made so close to 0 with the l2-norm that it compensates. OLS have certainly more spreaded weights, so when we use l2-norm, the important shrinkage induced by the square mitigate the difference between each estimator's weights, so their l2-norm is pretty close (if the weights are close to zero). For example, $1^2 - 0.05^2$ is higher than $0.8^2 - 0.3^2$ that in turn is higher than $0.6^2 - 0.1^2$.
But with l1-norm, the difference is stronger, we have not the "non linear" effect of the square that will shrink many of the weights close to zero even closer, reducing the difference between the estimator's weights that are closed to zero. l1-norm on OLS will add many positive quantities with now a bigger difference with the lasso weights, such that at the end, OLS l1-norm is particularly higher than Lasso, and now choosing this norm we better understand the regularization effects of the latter algorithm. It is also consistent with the fact that $\forall x \in \mathbb{R}^d, \lVert x \rVert_2 \leq \lVert x \rVert_1$.

Wa can also notice that the norm of the weights increases when $\alpha$ ($\lambda$ in the paper) increases, which is logical as bigger lambda means stronger penalization of the cost function. But it does not necessarily mean that the model becomes better and better. In many of our experiments, the best value for $\alpha$, regarding a range between 0 and 1 with 0.1 steps, is very often 0.1. So a bit of regularization is good, but too much can lead to a slight decrease of performances. This is the classical Bias-Variance Tradeoff of machine learning and statistics. We have to find the best balance between unerfitting and overfitting. Another fact that can explain a low value for $\alpha$ is the setting. Here, when you have so much features and hence a consequent number of weights, each one is multiplied by $\alpha$, such that increasing $\alpha$ lead to a greater value than if you had less weights. Imagine you have 10 weights, each equal to 1. Then 0.1 * 10 = 1, and 0.2 * 10 = 2, and 2 - 1 = 1. Now if all weights are still constant but now they are 10000, we now have respectively 1000, 2000, and a significative difference of 1000.

When we print the spreadness of the parameters vectors, it confirms our intuition. There are only few non-zero weights for the Lasso, such that the l2-norm of some big weights is close to the l2-norm of more non-zero weights a bit smaller, concavity of the square root clearly playing a role here. But now, if we only add the coordinates of the parameters vectors, the sparsity of the lasso vector yields to better results and much smaller norm than the OLS.

With 100 informative features :

Roughly the same.

With 1000 informative features :

Same.


With n_features = 1000

With 10 informative features :

OLS now performs better with 0.55 MSE, but Lasso is still better with 0.08 MSE at $\alpha = 0.1$. For the following value of $\alpha$, it is still better but after regularization is too strong and Lasso is worst than OLS.

So even with a small ratio of informative features / number of features, OLS is not horrible and not too far from Lasso. As there are only ten significant features, it is still logical that Lasso performs better with appropriate $\alpha$, but this advantage now very rapidly vanishes with $\alpha$ increasing. It is surely linked to the fact that with less features, even if the informative ones are not numerous, we have less risks of multicolinearity and also less risks for the OLS to overfit by using all the different features.

With 100 informative features :

Now MSE of OLS is a bit higher than the MSE of the Lasso. Again, it matches our understanding. As we have more informative features, the less we need to shrink some weights beacause of uninformative features. Hence wee need to have more weights that are non-zeros, and then to explore with more freedom the parameters space. The regularization is probabiliy to strong and prevent the lasso model to learn the usefull weights. To confirm this hypothesis, let's run the code again but with smaller values of $\alpha$, i.e with a smaller penalty. 
It is exactly what happens, the weights get bigger and now the MSE of Lasso is again less than for the OLS. We clearly see the tradeoff mechanism. Moreover, a fine-tuning of lambda (we're doing a kind of cross-validation here with our grid search) can always lead to an acceptable regularization. 


Note that with $\alpha = 0$ we have no regularization, so logically we get the same error. Sometimes, Lasso performs even worse than OLS. It can happen and it's not for a mathematical reason but an implementation reason. Sklearn warns us that even if it should be the same, $\alpha = 0$ is discouraged in LASSO because the implementation of the algorithm can then lead to far worse results and could fail to converge, as it is the case here.

In [ ]:
def compute_lipschitz(dico_estim_y):
    
    differences_y = []
    differences_estimators = []
    len_dico = len(dico_estim_y)

    for i in range(len_dico):
        
        yi = dico_estim_y[i][0] # we note y_i the ith vector of the dictionary but it's a vector of R^d, 
        # not necessarily a single coordinate  in general
        estimator_yi = dico_estim_y[i][1] # the coresponding estimator X Beta_hat(yi)
        
        for j in range(i + 1, len_dico):
            
            yj = dico_estim_y[j][0]
            estimator_yj = dico_estim_y[j][1]
            differences_y.append(MSE(yi, yj, squared=False))  # set squared = False to have the norm as in the paper, and not the norm squared
            differences_estimators.append(MSE(estimator_yi, estimator_yj, squared=False))
            
    return differences_y, differences_estimators

In [ ]:
# we now verify that the Lipschitz character for the estimator holds with L = 1

rng = default_rng(0)

n_samples = 1000
n_features = 100
n_informative = 10
bias = 1

noise = 2 # fixed noise now

alpha = 0.01

lasso = Lasso(alpha=alpha)

dico_estim_y = {}

nb_realizations = 10

f = None

dico_estim_y = {}

for i in range(nb_realizations):
    
    if not i:
        
        dataset = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative, bias=bias, noise=noise, 
                              coef=True, random_state = 0)

        dataset, coef = dataset[:2], dataset[2]

        lr = LinearRegression()
        lr.coef_ = coef
        lr.intercept_ = bias

        f = lr.predict(dataset[0])  # the mean vector, that we can compute thanks to the coef given by make_regression

        lasso.fit(dataset[0], dataset[1])

        predictions_lasso = lasso.predict(dataset[0])

        dico_estim_y[i] = [dataset[1], predictions_lasso]
        continue
        
    new_y = f + rng.normal(0, noise, n_samples)
    
    lasso.fit(dataset[0], new_y)
   
    predictions_lasso = lasso.predict(dataset[0])

    dico_estim_y[i] = [new_y, predictions_lasso]
    

differences_y, differences_estimators = compute_lipschitz(dico_estim_y)
axis = np.arange(len(differences_y))

plt.plot(axis, differences_y, label='diff y, y\'', color='tab:orange')
plt.plot(axis, differences_estimators, label='diff estimator_y, estimator_y\'', color='b')
plt.legend()
plt.title("The Lipschitz condition holds.")
plt.show()

In [ ]:
differences_estimators = np.array(differences_estimators)
differences_y = np.array(differences_y)

np.sum(differences_estimators > differences_y)

In [ ]:
np.sum(differences_estimators == differences_y)

In [ ]:
np.sum(differences_estimators < differences_y)

We are now going to illustrate **Theorem 3.2**. We will first generate different noises to obtain different vectors \mathbf{y}, and find a value $R$ for which we have the condition about the probability in this theorem that holds. Of course, a theoretical value for R is given in the paper, but it is linked to oracle inequalities and we are not sure if such a value of R is easy and fast to compute. So we decided for the moment to follow a less ambitious but safer path.

Once we have our $R$, we'll check that our results are consistent with the boundaries mentionned in the theorem. Be aware that we will compute an approximation of the probability involved, we don'use an exact expression, instead we approximate with an arbitrary (not too low) number of realizations of the gaussian noise.

In [ ]:
rng = default_rng(0)

n_samples = 1000
n_features = 100
n_informative = 10
bias = 1

noise = 2 # fixed noise now

alpha = 0.01

lasso = Lasso(alpha=alpha)

nb_realizations = 20

f = None

all_predictions = []

for i in range(nb_realizations):
    
    if not i:

        dataset = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative, bias=bias, noise=noise, 
                              coef=True, random_state = 0)
    
        dataset, coef = dataset[:2], dataset[2]
        
        lr = LinearRegression()
        lr.coef_ = coef
        lr.intercept_ = bias
    
        f = lr.predict(dataset[0])  # the mean vector, that we can compute thanks to the coef given by make_regression
        
        lasso.fit(*dataset)
    
        predictions_lasso = lasso.predict(dataset[0])
        all_predictions.append(predictions_lasso)
        
        continue
    
    new_noise = rng.normal(0, 2, n_samples)
    
    new_y = f + new_noise
    
    lasso.fit(dataset[0], new_y)

    predictions_lasso = lasso.predict(dataset[0])
    all_predictions.append(predictions_lasso)

all_predictions = np.array(all_predictions)
all_pred_errors = np.array([MSE(prediction, f, squared=False) for prediction in all_predictions])

# then, thanks to pandas, it's easy to have the median and so our R

print(f"Number of prediction errors computed : {all_pred_errors.shape[0]}\n")
median = pd.Series(all_pred_errors).median()

print(f"Median of these predictions errors (which gonna be our R) {median}\n")
print(f"We check the number of prediction errors <= median to see if it's correct : {np.sum(all_pred_errors <= median)}.\nIt's correct !")

In [ ]:
# before doing the computation, we need to write a function to compute the "empirical" probability, i.e ecdf

def compute_emp_proba(variable_values, x_values):  # return empirical probability of being inferior to some quantity x
    return np.mean(variable_values.reshape(variable_values.shape[0], 1) <= x_values, axis=0)
    


In [ ]:
# now we verify the given bound

R = median

t_max = (np.max(all_pred_errors) - R) * np.sqrt(n_samples) / noise  # we create meaningful values for t to well illustrate the bound
eps = 10e-8  # because t should not be zero, t > 0 
range_t = np.linspace(eps, t_max, 100)
x_values = R + range_t * noise / np.sqrt(n_samples)  # * noise because it is the std here

emp_probas = compute_emp_proba(all_pred_errors, x_values) # have 

plt.plot(range_t, norm.cdf(range_t, loc=0, scale=1), label="cdf N(0, 1)")
plt.plot(range_t, emp_probas, label="P(prediction error)")
plt.legend()
plt.title(f"First inequality of Theorem 3.2");

Again we see that the inferior bound always hold, except in a very tiny interval at the beginning but this perfectly makes sense since we are using the empirical cumulative distribution function. So when we increase R, i.e the median here, by a very small amount (a very little t), it does not change anything, whereas for the cumulative distribution function it changes, it indeed increases. (Sure the cdf in the specialized library we used is also discretized, everything is at the end in computer science, but the discretization granularity must be much higher).  

In the cell below, we see that the bound for the expectation is also verified.

In [ ]:
approx_error = np.mean(all_pred_errors)
print(f"Approximation of the expectation : {approx_error} <= {R + noise / np.sqrt(2 * np.pi * n_samples)} the bound given in the paper.")

We just verified that the bounds of **Theorem 3.2** perfectly hold.

As the theoretical bounds for oracle inequalities imply solving several constrained optimization problems, we kind ran out of time to illustrate this properly. Nevertheless, we let the snippet of code used to check condition on the event that plays a major role in all the section 4.

In [ ]:
p = dataset[0].shape[1]
original_noise = (dataset[1] - f)
lambda_param = 2 * noise * np.sqrt(2 * np.log2(p) / n_samples)
norm_max = (1 / n_samples) * np.max(dataset[0].T @ original_noise)
lambda_param / 2

print(f"Event of proposition 4.1 We check that the quantity in the left is inferior to lambda/2.\n{norm_max} <= {lambda_param / 2}. It's ok !")